In [ ]:
import os
import json
import csv
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
import jellyfish
import pandas as pd
import numpy as np
from pathlib import Path
from itertools import repeat


# A method to pull the data

In [ ]:
# def make_doc_details_values_list(input_directory,output_directory):
    
#     """
#     Pulls all of the naming existing on the verdicts jsons and stores them in sets.
    
#     Get the verdicts location as input directory and the output folder which will 
#     store the sets as txt file with the names seperated by '*'.
    
#     input_directory - path as a string
#     output_directory - path as a string
    
#     returnes all the sets created one by one.
#     """

#     # set initialization
#     judge_set = set()
#     petitioner_set = set()
#     defendse_set = set()
#     petitioner_attorney_set = set()
#     defendse_attorney_set = set()
    
#     # populating the sets
#     for verdict in os.scandir(input_directory):
#         with open(verdict,"r",encoding="utf-8") as verdict_file:
#             verdict_json = json.load(verdict_file)
            
#             judges = verdict_json["_source"]["doc"]["Doc Details"]["לפני"]
#             if(judges != None):
#                 for judge in judges:
#                     judge_set.add(judge)
            
#             petitioners = verdict_json["_source"]["doc"]["Doc Details"]["העותר"]
#             if(petitioners != None):
#                 for petitioner in petitioners:
#                     petitioner_set.add(petitioner)
            
#             defense = verdict_json["_source"]["doc"]["Doc Details"]["המשיב"]
#             if(defense != None):
#                 for defendee in defense:
#                     defendse_set.add(defendee)
                    
#             petitioner_attorneys = verdict_json["_source"]["doc"]["Doc Details"]["בשם העותר"]
#             if(petitioner_attorneys != None):
#                 for petitioner_attorney in petitioner_attorneys:
#                     petitioner_attorney_set.add(petitioner_attorney)
                    
#             defendse_attorney = verdict_json["_source"]["doc"]["Doc Details"]["בשם המשיב"]
#             if(defendse_attorney != None):
#                 for def_attorney in defendse_attorney:
#                     defendse_attorney_set.add(def_attorney)
    
#     # creating the files
#     cur_path = output_directory + "/" + 'judges.txt'
#     with open(cur_path,"w",encoding="utf-8") as judges_file:
#          for judge in judge_set:
#             judges_file.write(judge + "*")
        
#     cur_path = output_directory + "/" + 'petitioner.txt'
#     with open(cur_path,"w",encoding="utf-8") as petitioner_file:
#         for petitioner in petitioner_set:
#               petitioner_file.write(petitioner + "*")
        
#     cur_path = output_directory + "/" + 'defense.txt'
#     with open(cur_path,"w",encoding="utf-8") as defense_file:
#         for defense in defendse_set:
#             defense_file.write(defense + "*")
                
#     cur_path = output_directory + "/" + 'petitioner_attorneys.txt'
#     with open(cur_path,"w",encoding="utf-8") as petitioner_attorney_file:
#         for petitioner_attorney in petitioner_attorney_set:
#                 petitioner_attorney_file.write(petitioner_attorney + "*")
                
#     cur_path = output_directory + "/" + 'defendse_attorney.txt'
#     with open(cur_path,"w",encoding="utf-8") as defendse_attorney_file:
#         for defendse_attorney in defendse_attorney_set:
#             defendse_attorney_file.write(defendse_attorney + "*")
                
#     return judge_set,petitioner_set,defendse_set,petitioner_attorney_set,defendse_attorney_set
    

In [ ]:
# verdicts_path = 'C:/Users/Itai Ofir/HebrewCourtVerdictsAnalyzer/ML/data/splitteredjsonsaftercorrection'
# files_directory = 'C:/Users/Itai Ofir/HebrewCourtVerdictsAnalyzer/ML/data/normalization'
# a,b,c,d,e = make_doc_details_values_list(verdicts_path,files_directory)

# Creating the df

In [ ]:
def create_naming_df(path):
    
    """
    
    Read txt file with names serperated by '*'.
    
    Populating a pandas dataframe with the names as well as the role, for example: 
    אסתר חיות - judges
    
    input - the txt file path
    
    returns the newly created df
    
    """
    
    #check if path exists
    
    if os.path.exists(path):
        
        # initialize lists
        
        names = list()
        roles = list()
        
        # iterate on the files
        for file in os.scandir(path):
            
            # populate lists
            with open(file,"r",encoding="utf-8") as cur_file:
                
                # variable contains the number of names on the examined role
                before_assignment_count = len(names)
                
                # read the whole file
                cur_str = cur_file.read()
                
                # get only file name without the format
                cur_role = Path(file).name.split(".")[0]
                
                # append all the names to the current list
                names = names + (cur_str.split("*"))
                cur_list = list()
                
                # calculate how many names exists on the current iteration and extend roles list accoridngly
                cur_list.extend(repeat(cur_role,(len(names)-before_assignment_count)))
                roles = roles + cur_list
                print(str.format("{0} - {1}",cur_role,str(len(names)-before_assignment_count)))           
        
        # populate the df with the resulting lists
        df = pd.DataFrame()
        df["Before"] = names
        df["Role"] = roles
        
        return df
    
    else:
        return None

# Generic regex rephraser

In [ ]:
def eliminate_words_corresponds_to_regex_list(rgx_list, replace_with_that_str, input_text):
    
    """
    
    rgx_list - list of patterns
    
    replace_with_that_str - str to replace when match exists
    
    input_text - text to look on for patterns
    
    returns the new text after subtracting the matches
    
    """
    
    new_text = input_text
    for rgx_match in rgx_list:
        new_text = re.sub(rgx_match, replace_with_that_char, new_text)
    return new_text

# Regex rephraser for naming lists

In [ ]:
def apply_regex_rules_on_naming_csv(replace_with_this_str:str, input_name:str, csv_path:str):
    
    
    """
    
    Adds to every name hebrew prefixes that might exists on the data.
    
    This method is very expansive and not recommended when not necessary.
    
    
    
    replace_with_this_str - the str tobe replaced with
    
    input_name - name string
    
    csv_path - the csv that contains the names to be subtracted from each and every matched column
    
    
    returns the element after subtracting matched words
    
    """
    
    # initalization
    updated_name = input_name
    names_list=list()
    rgx_list=list()
    
    #open the csv file with the words to eliminate
    with open(csv_path, "r", encoding = 'utf-8') as csv_file:
        
        names_list = csv.reader(csv_file,delimiter=',')
                
        names = [row[0] for row in names_list]
        
        # add selcted prefixes
        for name in names:
            rgx_list.append(str.format("ו?"+name))
            rgx_list.append(str.format("ל?"+name))
            rgx_list.append(str.format("כ?ש?"+name))
            rgx_list.append(str.format("ה?"+name))
            rgx_list.append(str.format("ב?"+name))
            rgx_list.append(str.format("מ?"+name))
            rgx_list.append(str.format("ש?"+name))
            rgx_list.append(str.format("כ?"+name))
            rgx_list.append(name)
            
            if len(name.split()) > 1: 
                rgx_list.append(str.format(name.split()[0] + '-' + name.split()[1]))
                
            if len(name.split('-')) > 1:
                rgx_list.append(str.format(name.split('-')[0] + ' ' + name.split('-')[1]))
           
        for rgx_match in rgx_list:
            
            # take only specific words - not substrings, \\b is the word border.
            pattern = re.compile(str.format("\\b{0}\\b",rgx_match))
            updated_name = re.sub(pattern,replace_with_this_str,updated_df_element)
    
    after_elimination = ' '.join(updated_name.split())
    
    return after_elimination


# Eliminate empty and duplicates

In [ ]:
def eliminiate_empty_and_duplicates(df,column:str):
    
    """
    
    eliminates empty rows and duplicated rows of a specific column of the df
    
    df - Data Frame to work on
    
    column - column name
    
    returns the df after the percedure
    
    """
    
    new_df = df.copy()
   
    new_df[new_df[column].astype(bool)] 
    print(str.format('After eliminating empty {0}',new_df.shape))
    
    new_df.drop_duplicates(subset=column,keep=False,inplace=True)
    print(str.format('After clearing duplicates {0}',new_df.shape))
    
    new_df.reset_index(drop=True,inplace=False)
   
    return new_df

# Creating lewenstein distance based method to eliminate 2 chars apart string

In [ ]:
def is_identical(input_word:str,input_collection,mismatch_count):
    
    """
    
    check if specified word has mismatch_count amount of chars difference from any of the words in the collection
    
    input_word - string 
    
    input_collection - collection that stores strings to check upon
    
    mismatch_count - the maximal difference between the input word and the matched one
    
    retuns True for a match and False for a mismatch
    
    """
    
    for item in input_collection:
        if jellyfish.levenshtein_distance(word,item) <= mismatch_count:
            return True
    return False
    

# Eliminating verdic naming common uses

In [ ]:
def eliiminates_naming(text:str,path):
    
    """
    
    Eliminates all names match that has been on the csv file
    
    text - the string to work on
    
    path - the csv file path
    
    returns the string after eliminating all matched words
    
    """
    
    with open(path,"r",encoding="utf-8") as csv_name:
        csv_names = csv.reader(csv_name,delimiter=',')
        names = [row[0] for row in csv_names]
        after_names = ' '.join([word for word in str(text).split() if word not in names])
    
    return after_names

In [ ]:
# def eliiminates_naming(input_names:list,path):
    
#     """
    
#     Eliminates all names match that has been on the csv file
    
#     input_names - a list containing the strings to work on
    
#     path - the csv file path
    
#     returnes the new list after eliminating all matched words from the original strings
    
#     """
    
#     new_list = list()
#     with open(path,"r",encoding="utf-8") as csv_name:
#         csv_names = csv.reader(csv_name,delimiter=',')
#         names = [row[0] for row in csv_names]
#         for name in input_names:
#             after_names = ' '.join([word for word in str(name).split() if word not in names])
#             if after_names != '':
#                 new_list.append(after_names)
            
#     return new_list

# Eliminating unwanted chars

In [ ]:
def eliiminates_unwanted_chars(text):
    
    """
    
    That method is in charge of some of the pre-process precedures:
    
    1. eliminates pharenthesis  
    2. eliminate id numbers
    3. minimize spaces on last names
    
    text - the string we work on
    
    returns the text after the process.
    
    """
    
    for word in str(text).split(' '):
        if word.find("(") != -1 or word.find(")") != -1:
            text = text.replace(word,"")
        if word.find('.') != -1:
            for char in word:
                if char in string.digits:
                    text = text.replace(word,"")
                    break
                    
    text = '-'.join(text.split(' - '))
            
    text = ' '.join([word for word in text.split() if word != ' '])
        
    return text

# Handling multinames rows

In [ ]:
def split_df_to_proper_name_and_multiname(df,column:str):
    
    """
    
    splits given df to two:
    
    first part is names that has maximal value of 3, first name and two last names or less.
    
    second part are names that containes more than 3 strings
    
    df - the data frame to work on
    
    column - the string describing the desired column
    
    returns the original df as the first value and the new one as the second, 
    
    when the original now contains only "good" values and the second only "problematic" ones
    
    """
    
    # initialization
    df.reset_index(inplace=True,drop=True)
    indexes = list()
    new_values = list()
    
    # drop all the rows of the specified column
    df = df.dropna(subset=[column])
    
    # iterate on each row as string
    for i, presplit in enumerate(df[column].astype(str)):
        
        # split and check how many strings exists
        values = presplit.split(' ')
        if len(values) > 3:
            
            # append the index and the whole string
            indexes.append(i)
            new_values.append(presplit)
    
    # copy all the values that has been found as "problematic" ones to a new df
    new_df = df.iloc[indexes, :].copy()
    
    # populate the new df with the values
    new_df[column] = new_values
    
    # drop the problematic indexes from the original df and reset indexes on both of the dfs
    df.drop(df.index[indexes],inplace=True)
    df.reset_index(inplace=True,drop=True)
    new_df.reset_index(inplace=True,drop=True)
    
    return df,new_df

# Handling un-proper names

In [ ]:
def orginize_name(df,column:str):
    
    """
    
    deals with names rows that contain more the 3 strings
    
    the mechanism handles two secnarios sperately:
    
    1. when then amount of strings is precisely 3, 
       it fixes the first string to be the first name and both second and third to be last name seperated by '-'
    
    2. even amount of strings - iterate and add couple of strings as first name and last name
    
    assumptions:
    
    all names has already passed threw the preprocess - thus, last name that contains '-' is present without any spaces.
    
    there will be misidentified strings on that process, it relies on the parsed data.
    
    df - the data frame to work on
    
    column - the string describing the column 
    
    returns the same df after the process
    
    """
    
    # initialization
    new_values=list()
    indexes = list()
    
    # iterate on each row as string
    for i, presplit in enumerate(df[column].astype(str)):
        values = ' '.join(presplit.split()).split()
        
        # if the len is precisely 3, fix first name and two last names seperated by '-'
        if len(values) == 3:
            full_name = str()
            first_name = values[0]
            values.remove(values[0])
            last_name = '-'.join(values)
            if first_name.find('-') == -1:
                full_name = str.format('{0} {1}',first_name,last_name)
            else:
                full_name = str.format('{0} {1}',last_name,first_name)
            
            df.at[i,column] = full_name
        
        # if the len > 2 and even, iterate and assign couples until the string is empty 
        if len(values) > 2 and len(values) % 2 == 0:
            for idx in range(int(len(values)//2)):
                splitted=str()
                if values[0].find('-') == -1:
                    splitted = str.format("{0} {1}",values[2*idx],values[(2*idx)+1])
                
                # that scenario tries to handle cases when last names comes before first ofir-buzaglo itai instead of itai ofir-buzaglo
                else:
                    splitted = str.format("{0} {1}",values[2*idx+1],values[(2*idx)])
                
                # if the index equal to 0 the series already exists, just assign
                if idx == 0:
                    df.loc[i,column] = splitted
                
                # if the index is greated than 0, hence we must append new series.
                elif idx > 0:
                    cur_series = pd.Series(df.loc[i,:])
                    cur_series.at[column] = splitted
                    df.append(cur_series)
        
    return df

# Sort the attorney and judges by frequency and detrmine naming convention

In [ ]:
def count_normalized_names(df,path:str,roles:list):
    
    """
    
    counts all occurences of the names exists on the data frame after the preprocess in the whole available verdicts derictory
    
    assumptions:
    1. the right form of the name is majorly present

    df - the data frame to work on
    
    path - the path that holds all the vericts jsons available 
    
    roles - a list of roles to be examined.
            Attention! use precisely json scheme name here, for example: העותר
            
    returns the df after counters assignment
    """
    
    # initialization
    df["Count_Full_Name"] = 0
    df["Count_First_Name"] = 0
    df["Count_Last_Name"] = 0

    # count the amount of files present
    only_files = next(os.walk(path))[2]
    total=len(only_files)
    
    # iterate on the files
    for count,file in enumerate(os.scandir(path)):       
        with open(file,"r",encoding='utf-8') as json_file:
            verdict_json = json.load(json_file)
            for role in roles:
                
                # check for match and advance counter accordingly.
                # look for a match between all the roles given
        
                cur_verdict = ' '.join(verdict_json["_source"]["doc"]["Doc Details"][role])
                for idx,row in enumerate(df['Full_Name'].astype(str)):
                    if row in cur_verdict:
                        df.at[idx,'Count_Full_Name'] += 1
                        cur_verdict = cur_verdict.replace(row,'')
                    if not cur_verdict.strip():
                        break
                
                cur_verdict = ' '.join(verdict_json["_source"]["doc"]["Doc Details"][role])        
                for idx,row in enumerate(df['First_Name'].astype(str)):
                    if row in cur_verdict:
                        df.at[idx,'Count_First_Name'] += 1
                        cur_verdict = cur_verdict.replace(row,'')
                    if not cur_verdict.strip():
                        break
                
                cur_verdict = ' '.join(verdict_json["_source"]["doc"]["Doc Details"][role])        
                for idx,row in enumerate(df['Last_Name'].astype(str)):
                    if row in cur_verdict:
                        df.at[idx,'Count_Last_Name'] += 1
                        cur_verdict = cur_verdict.replace(row,'')
                    if not cur_verdict.strip():
                        break
        
        #print each 10000 files
        if count % 10000 == 0:
            print(str.format("{0}/{1}",count,total))
    
    
    return df

In [ ]:
def tidy_split(df, column, sep='|', keep=False):
    
    # credit: https://github.com/cognoma/genes/blob/721204091a96e55de6dcad165d6d8265e67e2a48/2.process.py#L61-L95
    
    """
    Split the values of a column and expand so the new DataFrame has one split
    value per row. Filters rows where the column is missing.

    Params
    ------
    df : pandas.DataFrame
        dataframe with the column to split and expand
    column : str
        the column to split and expand
    sep : str
        the string used to split the column's values
    keep : bool
        whether to retain the presplit value as it's own row

    Returns
    -------
    pandas.DataFrame
        Returns a dataframe with the same columns as `df`.
    """
    indexes = list()
    new_values = list()
    df = df.dropna(subset=[column])
    for i, presplit in enumerate(df[column].astype(str)):
        values = presplit.split(sep)
        if keep and len(values) > 1:
            indexes.append(i)
            new_values.append(presplit)
        for value in values:
            indexes.append(i)
            new_values.append(value)
    new_df = df.iloc[indexes, :].copy()
    new_df[column] = new_values
    new_df.reset_index(inplace=True,drop=True)
    return new_df

In [ ]:
path = 'C:/Users/Itai/HebrewCourtVerdictsAnalyzer/ML/data/normalization'
df = create_naming_df(path)
print(type(df))

In [ ]:
column='Full_Name'
df[column] =df["Before"].apply(lambda x :  eliiminates_unwanted_chars(x))
print("Eliminates unwanted" + str(df.shape))

print()
print(type(df))
print(df.describe())
print()

options = ['defense', 'petitioner']  
not_legal_df = df.loc[df['Role'].isin(options)]

print()
print(type(not_legal_df))
print(not_legal_df.describe())
print()

not_legal_index_names = df[ (df['Role'] == 'defense') | (df['Role'] == 'petitioner') ].index
df.drop(not_legal_index_names,inplace=True)

print()
print(type(df))
print(df.describe())
print()

df = tidy_split(df,column,sep=';')
print("after split by ; " + str(df.shape))

df = tidy_split(df,column,sep=',')
print("after split by , " + str(df.shape))

naming_path = 'C:/Users/Itai/HebrewCourtVerdictsAnalyzer/ML/naming_stopwords.csv'
stopwords_path = 'C:/Users/Itai/HebrewCourtVerdictsAnalyzer/ML/stopwordsafterfilter.csv'

df[column] = df[column].apply(lambda name_one : eliiminates_naming(name_one,naming_path))

df[column] = df[column].apply(lambda name_two : eliiminates_naming(name_two,stopwords_path))

df = orginize_name(df,column)
print(df.shape)
print(df.loc[:100,:])




In [ ]:
def append_first_and_last_name(df,column:str):
    
    """
    
    splits full name to first and last name
    
    df - the data frame to work on
    
    column - string describing the column name
    
    returns the df with new 'First_Name' and 'Last_Name' columns
    
    """
    
    #initalization
    indexes = []
    first_names = []
    last_names = []
    
    # iterate on each row and split, if first string contains '-', 
    # thus, it is last name and therefore we need to change posiotion
    for idx,name in enumerate(df[column].astype(str)):
        splitted = name.split()
        if len(splitted) == 2:
            indexes.append(idx)
            if splitted[0].find('-') == -1:
                first_names.append(splitted[0])
                last_names.append(splitted[1])
            else:
                first_names.append(splitted[1])
                last_names.append(splitted[0])
    
    # addign values to newly created column
    df.loc[indexes,'First_Name'] = first_names
    df.loc[indexes,'Last_Name'] = last_names
    
    print(str.format('Successfully splitted {0} names',len(indexes)))

In [ ]:
def check_if_name_exists(full_name:str,txt_path:str):
    
    """
    
    looks for a match with the supreme court judges txt file
    
    full_name - a string of the full name after the process
    
    text_path - a string with the txt file path
    
    
    """
    
    if len(full_name.split()) == 2:
        first_name = full_name.split()[0]
        last_name = full_name.split()[1]
        
        with open(txt_path,'r') as text_file:
            judges_list = text_file.readlines()
            for cur in judges_list:
                cur_first_name = cur.split()[0]
                cur_last_name = cur.split()[1]
                if cur_first_name[0] == first_name[0] and cur_last_name == last_name:
                    return cur
        
    return full_name

In [ ]:
def clean_single_name(name):
    
    new_name=str()
    
    if str(name).find('-') != -1:
        count = 0
        for char in name:
            if char == '-':
                count += 1
                
        if count != 1:
            left =str()
            right = str()
            left = name.split('-')[0]
            right = name.split('-')[1]
            left = ''.join([char for char in str(left) if char not in string.punctuation])
            right = ''.join([char for char in str(right) if char not in string.punctuation])
            new_name = str.format('{0}-{1}',left,right)    
    else:
        new_name = ''.join([char for char in str(name) if char not in string.punctuation])
    
    return new_name

In [ ]:
res = clean_single_name('it.\]['']ai')
print( res)

In [ ]:
append_first_and_last_name(df,'Full_Name')

In [ ]:
df['First_Name'] = df['First_Name'].apply(lambda first: clean_single_name(first))
df['Last_Name'] = df['Last_Name'].apply(lambda last: clean_single_name(last))

In [ ]:
csv_path = 'C:/Users/Itai/HebrewCourtVerdictsAnalyzer/ML/israel_supreme_court_judges.txt'
df['Normalized'] = df['Full_Name'].apply(lambda normalized : check_if_name_exists(normalized,csv_path))
df.loc[20:,'Full_Name']

In [ ]:
indexes = df[df['Normalized'] != np.nan]
print (indexes)

In [ ]:
print (df.loc[1400:1450,['First_Name','Last_Name','Normalized']])

In [ ]:
df.drop_duplicates(keep=False,inplace=True,ignore_index=True)
df.dropna()
df.reset_index(inplace=True,drop=True)
df.sort_values(by=['First_Name'],inplace=True)
csv_path = 'C:/Users/Itai/HebrewCourtVerdictsAnalyzer/ML/legal_by_first.csv'
df.to_csv(csv_path,index=False,encoding='utf-8')
df.sort_values(by=['Last_Name'],inplace=True)
df.reset_index(inplace=True,drop=True)
csv_path = 'C:/Users/Itai/HebrewCourtVerdictsAnalyzer/ML/legal_by_last.csv'
df.to_csv(csv_path,index=False,encoding='utf-8')

In [ ]:
def pre_process_not_legal(names):
    
    processed_names =[eliiminates_unwanted_chars(name) for name in names]
    
    return processed_names

In [ ]:
before = ['איתי אופיר - אבולעפיה','איתי (אופיר) אופיר','תז.235 משה אופיר']
after = pre_process_not_legal(before)
print(after)

In [ ]:
def split_by_char(names,char):
    
    splitted = list()
    
    
    for name in names:
        
        flag = False
        
        name_one = str()
        name_two = str()
        
        if name.find(char) != -1:
            
            name_one = ' '.join(name.split(char)[0].split())
            name_two = ' '.join(name.split(char)[1].split())
            
            flag=True
        
        if flag:
            splitted.append(name_one)
            splitted.append(name_two)
        else:
            splitted.append(name)
    
        
    return splitted

In [ ]:
def orginize_name(names):
    
    new_values=list()
    for name in names:
        values = ' '.join(name.split()).split()
        
        flag = False
        
        if len(values) == 3:
            first_name = values[0]
            values.remove(values[0])
            last_name = '-'.join(values)
            full_name = str.format('{0} {1}',first_name,last_name)
            new_values.append(full_name)
            
            flag = True
        
        elif len(values) > 2 and len(values) % 2 == 0:
            for idx in range(int(len(values)//2)):
                splitted = str.format("{0} {1}",values[2*idx],values[(2*idx)+1])
                new_values.append(splitted)
            flag = True
        
        if not flag:
            new_values = names
        
    return new_values

In [ ]:
def determine_order(full_name:str):
    
    """
    
    determines if specified full name is in the correct order: (first name) (last_name)
    
    full_name - string to be evaluated
    
    returns the string after order is set
    
    """
    
    # initalization
    new_str = str()
    
    # split the string
    values = full_name.split()
    first = values[0]
    last = values[1]
    
    # check if the first name contains '-' - that means it is last name in thw rong position
    if first.find('-') != -1:
        new_str = f'last first'
    else:
        new_str = f'first last'
    
    return new_str

In [ ]:
before1 = ['איתי  אופיר,אורן   חזן','אריק איינשטיין,גוזי כץ']
before2 = ['איתי  אופיר;אורן   חזן','אריק איינשטיין;גוזי כץ']
before3 = ['איתי אופיר']
after1 = split_by_char(before1,',')
after2 = split_by_char(before2,';')
after3 = split_by_char(before3,';')

print(after1)
print(after2)
print(after3)

In [ ]:
before1 = ['איתי אופיר אייזק ניוטון משה לוי','איתי אופיר אבולעפיה']
before2 = ['איתי-טוביה אופיר']

after1 = orginize_name(before1)
after2 = orginize_name(before2)

print(after1)
print(after2)

In [ ]:
def pre_process_legal(names):
    
    processed_names = list()
    
    names_after_first_split = split_by_char(names,';')
    names_after_second_split = split_by_char(names_after_first_split,',')
    
    naming_path = 'C:/Users/Itai/HebrewCourtVerdictsAnalyzer/ML/naming_stopwords.csv'
    stopwords_path = 'C:/Users/Itai/HebrewCourtVerdictsAnalyzer/ML/stopwordsafterfilter.csv'

    names_after_naming = [eliiminates_naming(name,naming_path) for name in names_after_second_split]
    names_after_stopwords = [eliiminates_naming(name,stopwords_path) for name in names_after_naming]

    names_ready_for_for_first_and_last_name = orginize_name(names_after_stopwords)
    after_single_clean_list = list()
    for name in names_ready_for_for_first_and_last_name:
        after_single_clean_list.append(clean_single_name(name))
    
    # add pd.Series addition to main DataFrame here.

In [ ]:
def write_normalized_values_to_json(verdict_path,dest_path,input_list,new_role_key):
    
    """
    
    adds the normalized key with the new values to the json and write it to destination
    
    verdict_path - the string of the verdict path
    
    dest_path - the string of the destination directory
    
    input_list - the normalized values strings in a list
    
    new_role_key - the key string that will be added to the json
    
    """
    
    with open(verdict_path,'r') as json_file:
        
        verdict = json.load(json_file)
        verdict['_source']['doc']['Doc Details'][role] = input_list
        verdict_id = verdict["_id"]
        path = dest_path + '/' + verdict_id
        
        with open(path,'w') as normalized_json:
            json.dump(verdict,normalized_json,ensure_ascii=False)
        

In [ ]:
def normalize(json_path,output_path):
    
    """
    
    takes the json path and path all the representitives names thru the precedure
    
    json_path - the verdict path string
    
    output_path - the string of the desired location to write the new json with the normalized names
    
    """
    
    cleaned_petitioners = list()
    cleaned_defedse = list()
    cleaned_judges = list()
    cleaned_petitioners_attorneys = list()
    cleaned_defendse_attorneys = list()
    
    with open(json_path,'r',encoding='utf-8')as json_file:
        
        verdict_json = json.load(json_file)
        
        petitioners = verdict_json["_source"]["doc"]["Doc Details"]["העותר"]
        defense = verdict_json["_source"]["doc"]["Doc Details"]["המשיב"]
        
        judges = verdict_json["_source"]["doc"]["Doc Details"]["לפני"]
        petitioner_attorneys = verdict_json["_source"]["doc"]["Doc Details"]["בשם העותר"]
        defendse_attorney = verdict_json["_source"]["doc"]["Doc Details"]["בשם המשיב"]
                
        for petitioner in petitioners:
            cleaned_petitioners.append(pre_process_not_legal(petitioner))
            
        for defender in defense:
            cleaned_defedse.append(pre_process_not_legal(defendee))
        
        for judge in judges:
            cleaned_judges.append(pre_process_legal(judge))
        
        for petitioner_attorney in petitioner_attorneys:
            cleaned_petitioners_attorneys.append(pre_process_legal(petitioner_attorney))
            
        for defendse_attorney in cleaned_defendse_attorneys:
            cleaned_defendse_attorneys.append(pre_process_legal(defendse_attorney))
            
        write_normalized_values_to_json(json_path,output_path,cleaned_petitioners,'העותר מנורמל')
        write_normalized_values_to_json(json_path,output_path,cleaned_defedse,'המשיב מנורמל')
        write_normalized_values_to_json(json_path,output_path,cleaned_judges,'לפני מנורמל')
        write_normalized_values_to_json(json_path,output_path,cleaned_petitioners_attorneys,'בשם העותר מנורמל')
        write_normalized_values_to_json(json_path,output_path,cleaned_defendse_attorneys,'בשם המשיב מנורמל')

In [ ]:
# df = df.append(not_legal_df,ignore_index=True)
# print(df.shape)
# df.reset_index(inplace=True,drop=True)
# print(df.loc[:50,:])


In [ ]:
# Testing counter

In [ ]:
# df.sort_values(by=['After'],ascending=False,inplace=True)
# df.reset_index(inplace=True,drop=True)


In [ ]:
# shrinked_df = df.sample(10)
# path = 'C:/Users/Itai/HebrewCourtVerdictsAnalyzer/ML/data/splitteredjsonsaftercorrection'
# names = ["בשם העותר","בשם המשיב","לפני","העותר","המשיב"]
# df = count_normalized_names(shrinked_df,column,path,names)
# csv_path = 'C:/Users/Itai/HebrewCourtVerdictsAnalyzer/ML/normalized_function.csv'
# df.to_csv(csv_path,index=False,encoding='utf-8')